In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 

import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import os 
import warnings 
warnings.simplefilter('ignore') 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator 
import matplotlib.pyplot as plt
%matplotlib inline

pd.plotting.register_matplotlib_converters() 

import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 
plt.style.use('dark_background') 
from sklearn.model_selection import train_test_split

import spacy
import string
pd.options.mode.chained_assignment = None

<h1 style="font-family: 'Lucida Sans', 'Lucida Sans Regular', 'Lucida Grande', 'Lucida Sans Unicode', Geneva, Verdana, sans-serif; text-align: center; color: #3A405A;">📊 Spacy & CBoW on Top world news </h1>
<h3 style="font-family: 'Lucida Sans', 'Lucida Sans Regular', 'Lucida Grande', 'Lucida Sans Unicode', Geneva, Verdana, sans-serif; text-align: center; color: #6B6E70;">An In-depth Guide</h3>

In [2]:

MainDataset = pd.read_csv("../input/top-25-world-news-2018-2023/WorldNewsData.csv") 

MainDataset 


,Date,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,"May 01, 2018","North Korea to open its sky, South Korean medi...",The Mueller probe ain't ending anytime soon,BRAND NEW: 2018 – Renault Alpine A110 – Start-...,The story of the Midwestern innocent let loose...,Mattis tempers optimistic outlook on Afghanist...,Facebook is taking on Tinder with new dating f...,OECD chief says Greece deserves debt relief af...,Facebook to launch dating service,U.N. could help Myanmar gather evidence of cri...,...,Iran's Nuclear Threat,Jean 'Binta' Breeze Performance At Dis Poem: W...,Migrant says he ‘accidentally’ raped a Belgian...,My Email Newsletter of Curated Marketing Content,Mueller asks for 2 more months before Flynn se...,Mueller asks for 2 more months before Flynn se...,Canada to tack a carbon price of at least 11 c...,Kanye West: 'The class war is one of the reaso...,Michael Stewart/WireImage Kylie Jenner simply ...,For first time health ministry will regulate d...
1,"May 02, 2018",'What have they got to fear? ' Trump threatens...,"Best Garage Heater Reviews incuding electric, ...",Apple just fueled everyone's biggest fear abou...,Google accused of using GDPR to impose unfair ...,Man died accidentally after climbing behind wo...,When the coveted Palme d’Or trophy is awarded ...,"""Borders are Illegal?"" Tucker Carlson Vs Dumb ...","Mark Zuckerberg says it will take 3 years to ""...",The 8 Best Cooling Pillows You Can Buy,...,Palestinians said set to withdraw recognition ...,Half the Australian businesses that got a tax ...,Ideas To Make College Work For You,House Republicans nominate Donald Trump for No...,UK's May faces local election losses as key Br...,Whynter ICM-15LS Ice Cream Maker Review,"Abbas says Jews’ behavior, not anti-Semitism, ...",House Republicans nominate Trump for Nobel Pea...,"Tyson Foods the largest U.S. meat processor, h...",Questions linger about how Melania Trump score...
2,"May 03, 2018",Hollywood isn’t simply churning out crummy rem...,Iran's foreign minister: US has ‘consistently ...,Rihanna covers the June 2018 problem of Vogue ...,U.S. soldiers are secretly fighting Saudi Arab...,Canadian MP Bob Saroya took trip to China paid...,2017 Best Black Friday and Cyber Monday Mattre...,"NBC, ABC Say Trump Lawyer Michael Cohen Was No...",South Korean janitor may keep gold bars found ...,Last chance: UN warns Cambodia over failing de...,...,The world’s last male northern white rhino has...,Uruguay's soybean crop forecasted to drop 43% ...,We Now Have A Working Nuclear Reactor for Othe...,Marketing’s Customer Obsession Imperative | C3...,Ex-Trump aide Michael Caputo warns: Mueller Ru...,11-Year-Old Girl With Autism Wouldn’t Talk In ...,Richard Corkery/NY Each day Information throug...,What Makes the Brand Stand Out?,UK drones in Syria using controversial 'vacuum...,"High fruit diet could help women conceive, stu..."
3,"May 04, 2018",Donald Trump says London hospital is 'like a w...,German seat on UN Security Council likely as I...,Gang Used Drone Swarm To Thwart FBI Hostage Raid,How Many Carbs Should You Eat Per Day To Lose ...,Russian Troll Farm Goes After Mueller’s Files,Israeli soldiers singing in Persian for Irania...,Labor Day 2017 Sales on Memory Foam Mattresses...,"How a Chinese city turned all its 16,000 buses...",Jamie Acourt arrested in Spain over suspected ...,...,"2013 Cadillac CTS Luxury Walkaround, Start up,...","UK Local elections: 4,000 people turned away f...",Nico Tortorella speaks onstage throughout Risi...,How to Increase Sales by Encouraging Mobile Sp...,China’s schooling Trump on the art of the delay,Abi Ofarim ist tot,The 8 Best Snow Boots For Women,Kerry quietly seeking to salvage Iran deal he ...,Trump's Syria aid freeze hits 'White Helmets' ...,US slams Turkey over population transfers in A...
4,"May 05, 2018",Missing teen’s mom implores Canada to take not...,William and Kate release new pictures of Princ...,MEGHANS BROTHER SEND

<div style="background-color: #E8EAF6; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h2 style="font-family: 'Verdana'; color: #3A405A;">🔍 Insights</h2>
    <p style="font-size: 18px; font-family: 'Verdana'; color: #3A405A; line-height: 1.5em;">When we have a dataset with similar data types in multiple columns, one approach is to merge all the columns. This often occurs with both image and text based dataset. </p>
</div>

In [3]:

Path_to_the_data = '../input/top-25-world-news-2018-2023/WorldNewsData.csv' 

df = pd.read_csv( Path_to_the_data, parse_dates=['Date'])
df


,Date,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2018-05-01,"North Korea to open its sky, South Korean medi...",The Mueller probe ain't ending anytime soon,BRAND NEW: 2018 – Renault Alpine A110 – Start-...,The story of the Midwestern innocent let loose...,Mattis tempers optimistic outlook on Afghanist...,Facebook is taking on Tinder with new dating f...,OECD chief says Greece deserves debt relief af...,Facebook to launch dating service,U.N. could help Myanmar gather evidence of cri...,...,Iran's Nuclear Threat,Jean 'Binta' Breeze Performance At Dis Poem: W...,Migrant says he ‘accidentally’ raped a Belgian...,My Email Newsletter of Curated Marketing Content,Mueller asks for 2 more months before Flynn se...,Mueller asks for 2 more months before Flynn se...,Canada to tack a carbon price of at least 11 c...,Kanye West: 'The class war is one of the reaso...,Michael Stewart/WireImage Kylie Jenner simply ...,For first time health ministry will regulate d...
1,2018-05-02,'What have they got to fear? ' Trump threatens...,"Best Garage Heater Reviews incuding electric, ...",Apple just fueled everyone's biggest fear abou...,Google accused of using GDPR to impose unfair ...,Man died accidentally after climbing behind wo...,When the coveted Palme d’Or trophy is awarded ...,"""Borders are Illegal?"" Tucker Carlson Vs Dumb ...","Mark Zuckerberg says it will take 3 years to ""...",The 8 Best Cooling Pillows You Can Buy,...,Palestinians said set to withdraw recognition ...,Half the Australian businesses that got a tax ...,Ideas To Make College Work For You,House Republicans nominate Donald Trump for No...,UK's May faces local election losses as key Br...,Whynter ICM-15LS Ice Cream Maker Review,"Abbas says Jews’ behavior, not anti-Semitism, ...",House Republicans nominate Trump for Nobel Pea...,"Tyson Foods the largest U.S. meat processor, h...",Questions linger about how Melania Trump score...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...,Iran's foreign minister: US has ‘consistently ...,Rihanna covers the June 2018 problem of Vogue ...,U.S. soldiers are secretly fighting Saudi Arab...,Canadian MP Bob Saroya took trip to China paid...,2017 Best Black Friday and Cyber Monday Mattre...,"NBC, ABC Say Trump Lawyer Michael Cohen Was No...",South Korean janitor may keep gold bars found ...,Last chance: UN warns Cambodia over failing de...,...,The world’s last male northern white rhino has...,Uruguay's soybean crop forecasted to drop 43% ...,We Now Have A Working Nuclear Reactor for Othe...,Marketing’s Customer Obsession Imperative | C3...,Ex-Trump aide Michael Caputo warns: Mueller Ru...,11-Year-Old Girl With Autism Wouldn’t Talk In ...,Richard Corkery/NY Each day Information throug...,What Makes the Brand Stand Out?,UK drones in Syria using controversial 'vacuum...,"High fruit diet could help women conceive, stu..."
3,2018-05-04,Donald Trump says London hospital is 'like a w...,German seat on UN Security Council likely as I...,Gang Used Drone Swarm To Thwart FBI Hostage Raid,How Many Carbs Should You Eat Per Day To Lose ...,Russian Troll Farm Goes After Mueller’s Files,Israeli soldiers singing in Persian for Irania...,Labor Day 2017 Sales on Memory Foam Mattresses...,"How a Chinese city turned all its 16,000 buses...",Jamie Acourt arrested in Spain over suspected ...,...,"2013 Cadillac CTS Luxury Walkaround, Start up,...","UK Local elections: 4,000 people turned away f...",Nico Tortorella speaks onstage throughout Risi...,How to Increase Sales by Encouraging Mobile Sp...,China’s schooling Trump on the art of the delay,Abi Ofarim ist tot,The 8 Best Snow Boots For Women,Kerry quietly seeking to salvage Iran deal he ...,Trump's Syria aid freeze hits 'White Helmets' ...,US slams Turkey over population transfers in A...
4,2018-05-05,Missing teen’s mom implores Canada to take not...,William and Kate release new pictures of Princ...,MEGHANS BROTHER SENDS PRINCE HARRY A SEC


<h2 style="font-family: 'Lucida Sans', 'Lucida Sans Regular', 'Lucida Grande', 'Lucida Sans Unicode', Geneva, Verdana, sans-serif; text-align: center; color: #3A405A;">Data Cleaning and Preprocessing </h2>




<div style="background-color: #E8EAF6; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <p style="font-size: 18px; font-family: 'Verdana'; color: #3A405A; line-height: 1.5em;">Having generated a corpus, you now need to take some steps to make sure that your texts are in a form that a computer can understand and work with. ‘Pre-processing’ is a catch-all term used for the different activities that you undertake to get your documents ready to be analysed. You may only use a few pre-processing techniques, or you may decide to use a wide array, depending on your documents, the kind of text you have and the kinds of analyses you want to perform.</p>
        
   <p style="font-size: 18px; font-family: 'Verdana'; color: #3A405A; line-height: 1.5em;">The first pre-processing step in any TDM project is to identify the cleaning that will need to be done to enable your analysis. Cleaning refers to steps that you take to standardise your text and to remove text and characters that aren’t relevant. After performing these steps, you'll be left with a nice ‘clean’ text dataset that is ready to be analysed.</p>
</div>




In [4]:
cols = df.columns.difference(['Date']) 

df['TopNews'] = df[cols].apply(lambda x: ', '.join(x.dropna().astype(str)), 1) 

df = df.drop(cols, axis=1)  

df

,Date,TopNews
0,2018-05-01,"North Korea to open its sky, South Korean medi..."
1,2018-05-02,'What have they got to fear? ' Trump threatens...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...
3,2018-05-04,Donald Trump says London hospital is 'like a w...
4,2018-05-05,Missing teen’s mom implores Canada to take not...
...,...,...
1854,2023-04-26,Uyghur death in Thai refugee detention center ...
1855,2023-04-27,Woman whose accusation led to the lynching of ...
1856,2023-04-28,An Ominous Heating Event Is Unfolding in the O...
1857,2023-04-29,Russian Occupied Ukrainian City Under 'Intense...


<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h2 style="font-family: 'Verdana'; color: #FF5733;"> Text Cleaning </h2>
</div>

In [5]:
# to lowercase 
df["TopNews_LowerCase"] = df["TopNews"].str.lower()  

df.head()

,Date,TopNews,TopNews_LowerCase
0,2018-05-01,"North Korea to open its sky, South Korean medi...","north korea to open its sky, south korean medi..."
1,2018-05-02,'What have they got to fear? ' Trump threatens...,'what have they got to fear? ' trump threatens...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...,hollywood isn’t simply churning out crummy rem...
3,2018-05-04,Donald Trump says London hospital is 'like a w...,donald trump says london hospital is 'like a w...
4,2018-05-05,Missing teen’s mom implores Canada to take not...,missing teen’s mom implores canada to take not...


<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h2 style="font-family: 'Verdana'; color: #FF5733;">  Removing the Punctutaions  </h2>
</div>

In [6]:

df.drop(["TopNews_LowerCase"], axis=1, inplace=True)  

PUNCT_TO_REMOVE = string.punctuation 

def remove_punctuation(text): 
    
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE)) 

df["Text-Without-Punctuation"] = df["TopNews"].apply(lambda text: remove_punctuation(text))

df.head()


,Date,TopNews,Text-Without-Punctuation
0,2018-05-01,"North Korea to open its sky, South Korean medi...",North Korea to open its sky South Korean media...
1,2018-05-02,'What have they got to fear? ' Trump threatens...,What have they got to fear Trump threatens to...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...,Hollywood isn’t simply churning out crummy rem...
3,2018-05-04,Donald Trump says London hospital is 'like a w...,Donald Trump says London hospital is like a wa...
4,2018-05-05,Missing teen’s mom implores Canada to take not...,Missing teen’s mom implores Canada to take not...


<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h2 style="font-family: 'Verdana'; color: #FF5733;"> Removal of the Stop Words </h2>

</div>

In [7]:

from nltk.corpus import stopwords 

", ".join(stopwords.words('english'))


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [8]:

STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text): 
    
    return " ".join([word for word in str(text).split() if word not in STOPWORDS]) 

df["Text-Without-Stop"] = df["Text-Without-Punctuation"].apply(lambda text: remove_stopwords(text)) 

df.head()


,Date,TopNews,Text-Without-Punctuation,Text-Without-Stop
0,2018-05-01,"North Korea to open its sky, South Korean medi...",North Korea to open its sky South Korean media...,North Korea open sky South Korean media report...
1,2018-05-02,'What have they got to fear? ' Trump threatens...,What have they got to fear Trump threatens to...,What got fear Trump threatens declassify FBI J...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...,Hollywood isn’t simply churning out crummy rem...,Hollywood isn’t simply churning crummy remakes...
3,2018-05-04,Donald Trump says London hospital is 'like a w...,Donald Trump says London hospital is like a wa...,Donald Trump says London hospital like war zon...
4,2018-05-05,Missing teen’s mom implores Canada to take not...,Missing teen’s mom implores Canada to take not...,Missing teen’s mom implores Canada take notes ...


In [9]:
from collections import Counter 

cnt = Counter()
for text in df["Text-Without-Punctuation"].values: 
    for word in text.split(): 
        cnt[word] += 1 
        
cnt.most_common(25)
         

[('to', 15953),
 ('in', 12583),
 ('of', 11083),
 ('the', 10658),
 ('and', 6521),
 ('for', 6130),
 ('a', 6109),
 ('on', 5595),
 ('US', 3560),
 ('as', 3207),
 ('The', 3123),
 ('is', 2990),
 ('with', 2943),
 ('by', 2718),
 ('at', 2517),
 ('from', 2405),
 ('after', 2335),
 ('Trump', 2237),
 ('says', 2148),
 ('Ukraine', 1908),
 ('over', 1638),
 ('Russia', 1545),
 ('that', 1523),
 ('China', 1522),
 ('has', 1476)]

<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h2 style="font-family: 'Verdana'; color: #FF5733;"> Removal of Frequent Words</h2>

</div>

In [10]:

FREQWORD = set([w for (w, wc) in cnt.most_common(10)])  

def remove_freqwords(text):
    
    return " ".join([word for word in str(text).split() if word not in FREQWORD])

df["TopNews_Without_Frequent"] = df["Text-Without-Stop"].apply(lambda text: remove_freqwords(text))

df.head()


,Date,TopNews,Text-Without-Punctuation,Text-Without-Stop,TopNews_Without_Frequent
0,2018-05-01,"North Korea to open its sky, South Korean medi...",North Korea to open its sky South Korean media...,North Korea open sky South Korean media report...,North Korea open sky South Korean media report...
1,2018-05-02,'What have they got to fear? ' Trump threatens...,What have they got to fear Trump threatens to...,What got fear Trump threatens declassify FBI J...,What got fear Trump threatens declassify FBI J...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...,Hollywood isn’t simply churning out crummy rem...,Hollywood isn’t simply churning crummy remakes...,Hollywood isn’t simply churning crummy remakes...
3,2018-05-04,Donald Trump says London hospital is 'like a w...,Donald Trump says London hospital is like a wa...,Donald Trump says London hospital like war zon...,Donald Trump says London hospital like war zon...
4,2018-05-05,Missing teen’s mom implores Canada to take not...,Missing teen’s mom implores Canada to take not...,Missing teen’s mom implores Canada take notes ...,Missing teen’s mom implores Canada take notes ...


<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h2 style="font-family: 'Verdana'; color: #FF5733;">  The Removal of the Rare Words in the opposit of the above  </h2>
</div>

In [11]:

# Drop the tow columns which are no more needed 
df.drop(["Text-Without-Punctuation", "Text-Without-Stop"], axis=1, inplace=True) 

n_rare_words = 10  

RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])  

def remove_rarewords(text): 
    
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])


df["text_without_rare_words"] = df["TopNews_Without_Frequent"].apply(lambda text: remove_rarewords(text)) 

df.head()


,Date,TopNews,TopNews_Without_Frequent,text_without_rare_words
0,2018-05-01,"North Korea to open its sky, South Korean medi...",North Korea open sky South Korean media report...,North Korea open sky South Korean media report...
1,2018-05-02,'What have they got to fear? ' Trump threatens...,What got fear Trump threatens declassify FBI J...,What got fear Trump threatens declassify FBI J...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...,Hollywood isn’t simply churning crummy remakes...,Hollywood isn’t simply churning crummy remakes...
3,2018-05-04,Donald Trump says London hospital is 'like a w...,Donald Trump says London hospital like war zon...,Donald Trump says London hospital like war zon...
4,2018-05-05,Missing teen’s mom implores Canada to take not...,Missing teen’s mom implores Canada take notes ...,Missing teen’s mom implores Canada take notes ...




<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h3 style="font-family: 'Verdana'; color: #FF5733;">  Stemming </h3>
    <p>
    The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. For instance:

am, are, is $\Rightarrow$ be <br>
car, cars, car's, cars' $\Rightarrow$ car <br>
The result of this mapping of text will be something like: <br>
the boy's cars are different colors $\Rightarrow$ <br>
the boy car be differ color <br>
    

</div>



In [12]:

from nltk.stem.porter import PorterStemmer 

# Drop the two columns 
df.drop(["TopNews_Without_Frequent", "text_without_rare_words"], axis=1, inplace=True) 

stemmer = PorterStemmer()

def stem_words(text): 
    
    return " ".join([stemmer.stem(word) for word in text.split()]) 

df["text_stemmed"]= df["TopNews"].apply(lambda text: stem_words(text))

df.head()


,Date,TopNews,text_stemmed
0,2018-05-01,"North Korea to open its sky, South Korean medi...","north korea to open it sky, south korean media..."
1,2018-05-02,'What have they got to fear? ' Trump threatens...,'what have they got to fear? ' trump threaten ...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...,hollywood isn’t simpli churn out crummi remak ...
3,2018-05-04,Donald Trump says London hospital is 'like a w...,donald trump say london hospit is 'like a war ...
4,2018-05-05,Missing teen’s mom implores Canada to take not...,miss teen’ mom implor canada to take note from...


<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h3 style="font-family: 'Verdana'; color: #FF5733;"> Lemmatization </h3>
    <p>
        We use lemmatization to convert the input data set into a form that chops off the grammatical syntax to simplify the data set. This implies that multiple different words in the raw data set could map onto the same word after lemmatization. Consequently, this reduces noise and simplifies the training process alongside the obvious increase in the speed of analysis.
    </p>
</div>



In [13]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [14]:
from nltk.stem import WordNetLemmatizer  

# initilizing the method 
lemmatizer = WordNetLemmatizer() 

def lemmatize_word(text): 
    
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()]) 

df["text_lemmatized"] = df["text_stemmed"].apply(lambda text: lemmatize_word(text)) 

df.head()


,Date,TopNews,text_stemmed,text_lemmatized
0,2018-05-01,"North Korea to open its sky, South Korean medi...","north korea to open it sky, south korean media...","north korea to open it sky, south korean mediu..."
1,2018-05-02,'What have they got to fear? ' Trump threatens...,'what have they got to fear? ' trump threaten ...,'what have they got to fear? ' trump threaten ...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...,hollywood isn’t simpli churn out crummi remak ...,hollywood isn’t simpli churn out crummi remak ...
3,2018-05-04,Donald Trump says London hospital is 'like a w...,donald trump say london hospit is 'like a war ...,donald trump say london hospit is 'like a war ...
4,2018-05-05,Missing teen’s mom implores Canada to take not...,miss teen’ mom implor canada to take note from...,miss teen’ mom implor canada to take note from...


<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h3 style="font-family: 'Verdana'; color: #FF5733;"> We want to provide the POS tag along the word for when it is lemmatized  :</h3>
    <p style="font-size: 18px; font-family: 'Verdana'; color: #FF5733; line-height: 1.5em;">
        Part-of-speech (POS) tagging, also called grammatical tagging, is the automatic assignment of part-of-speech tags to words in a sentence [9,10,11]. A POS is a grammatical classification that commonly includes verbs, adjectives, adverbs, nouns, etc
    </p>
    <p style="font-size: 18px; font-family: 'Verdana'; color: #FF5733; line-height: 1.5em;">
        First we do the POS tagging then we do the lemmatization on that!
    </p>
</div>

In [15]:
import nltk
from nltk.corpus import wordnet 
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()

wordnet_map = {"N": wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

def lemmatize_words(text): 
    
    pos_tagged_text = nltk.pos_tag(text.split())  
    
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text]) 

df["text_lemmatized"] = df["text_stemmed"].apply(lambda text: lemmatize_words(text)) 

df.head()


,Date,TopNews,text_stemmed,text_lemmatized
0,2018-05-01,"North Korea to open its sky, South Korean medi...","north korea to open it sky, south korean media...","north korea to open it sky, south korean mediu..."
1,2018-05-02,'What have they got to fear? ' Trump threatens...,'what have they got to fear? ' trump threaten ...,'what have they get to fear? ' trump threaten ...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...,hollywood isn’t simpli churn out crummi remak ...,hollywood isn’t simpli churn out crummi remak ...
3,2018-05-04,Donald Trump says London hospital is 'like a w...,donald trump say london hospit is 'like a war ...,donald trump say london hospit be 'like a war ...
4,2018-05-05,Missing teen’s mom implores Canada to take not...,miss teen’ mom implor canada to take note from...,miss teen’ mom implor canada to take note from...


<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h3 style="font-family: 'Verdana'; color: #FF5733;"> Removing URL and Emojies</h3>
</div>

In [16]:
# we use apply for the dataframe text! 
import re
from tqdm import tqdm

def remove_urls(text): 
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_emojis(text): 
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emojilar
                               u"\U0001F300-\U0001F5FF"  # simvollar va diagrammalar
                               u"\U0001F680-\U0001F6FF"  # transport va turli joylar
                               u"\U0001F1E0-\U0001F1FF"  # davlat bayroqlari
                               u"\U00002702-\U000027B0"  # dingbats
                               u"\U000024C2-\U0001F251"  # alamatchilik belgilari
                               "]+", flags=re.UNICODE) 
    return emoji_pattern.sub(r'',text)

# we apply these functions to each columns of the dataframe we want 

In [17]:
df['text_lemmatized'] = df['text_lemmatized'].apply(lambda x: remove_urls(x))

df['text_lemmatized'] = df['text_lemmatized'].apply(lambda x: remove_emojis(x))


In [35]:
df.head()

,Date,TopNews,text_stemmed,text_lemmatized
0,2018-05-01,"North Korea to open its sky, South Korean medi...","north korea to open it sky, south korean media...","north korea to open it sky, south korean mediu..."
1,2018-05-02,'What have they got to fear? ' Trump threatens...,'what have they got to fear? ' trump threaten ...,'what have they get to fear? ' trump threaten ...
2,2018-05-03,Hollywood isn’t simply churning out crummy rem...,hollywood isn’t simpli churn out crummi remak ...,hollywood isn’t simpli churn out crummi remak ...
3,2018-05-04,Donald Trump says London hospital is 'like a w...,donald trump say london hospit is 'like a war ...,donald trump say london hospit be 'like a war ...
4,2018-05-05,Missing teen’s mom implores Canada to take not...,miss teen’ mom implor canada to take note from...,miss teen’ mom implor canada to take note from...


<h1 style="font-family: 'Lucida Sans', 'Lucida Sans Regular', 'Lucida Grande', 'Lucida Sans Unicode', Geneva, Verdana, sans-serif; text-align: center; color: #6B6E70;">Working with SpaCy:</h1>

<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <p style="font-size: 18px; font-family: 'Verdana'; color: #FF5733; line-height: 1.5em;">
        spaCy is an open-source library for NLP in Python written in Cython. spaCy is designed to make it easy to build systems for information extraction or general-purpose natural language processing.
      </p>
</div>

In [18]:
import spacy
import random
from collections import Counter #for counting
import seaborn as sns #for visualization
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('seaborn')
sns.set(font_scale=2)
import json
def pretty_print(pp_object):
    print(json.dumps(pp_object, indent=2))
    
from IPython.display import Markdown, display
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))


In [19]:
!pip install --upgrade pip
!pip install textacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 6.7 MB/s eta 0:00:00
  Obtaining dependency information for floret~=0.10.0 from https://files.pythonhosted.org/packages/68/94/5f623c436f3ccf9fcf73fead49be54f28fb392ec9189d2e08211727fa8d9/floret-0.10.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for jellyfish>=0.8.0 from https://files.pythonhosted.org/packages/ac/5a/fafb2fe555f34e5aeed8c11153257c5af09197451eecb36207e4e2973aed/jellyfish-1.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.3/320.3 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [20]:
!python -m spacy download en_core_web_lg

nlp = spacy.load('en_core_web_lg')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l


<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h3 style="font-family: 'Verdana'; color: #FF5733;">Extracting and Explaining the Entities with Spacy:</h3>
    <p style="font-size: 18px; font-family: 'Verdana'; color: #FF5733; line-height: 1.5em;"> Named-entity recognition (NER) is the process of locating named entities in unstructured text and then classifying them into predefined categories, such as person names, organizations, locations, monetary values, percentages, and time expressions.You can use NER to learn more about the meaning of your text. For example, you could use it to populate tags for a set of documents in order to improve the keyword search. You could also use it to categorize customer support tickets into relevant categories.spaCy has the property .ents on Doc objects. You can use it to extract named entities:</p>
</div>



In [21]:

def explain_text_entities(text):
    doc = nlp(text)
    for ent in doc.ents:
        print(f'Entity: {ent}, Label: {ent.label_}, {spacy.explain(ent.label_)}')


In [22]:
explain_text_entities(df['TopNews'][9])

Entity: Japan, Label: GPE, Countries, cities, states
Entity: China, Label: GPE, Countries, cities, states
Entity: a decade, Label: DATE, Absolute or relative dates or periods
Entity: two, Label: CARDINAL, Numerals that do not fall under another type
Entity: Asian, Label: NORP, Nationalities or religious or political groups
Entity: Electric Food Slicer Reviewed, Label: ORG, Companies, agencies, institutions, etc.
Entity: HACHE/AFP/Getty Photos, Label: ORG, Companies, agencies, institutions, etc.
Entity: the Cannes Movie Pageant, Label: EVENT, Named hurricanes, battles, wars, sports events, etc.
Entity: Bella Hadid, Label: PERSON, People, including fictional
Entity: Magnum, Label: PERSON, People, including fictional
Entity: Take Pleasure Severely”, Label: WORK_OF_ART, Titles of books, songs, etc.
Entity: Thousands, Label: CARDINAL, Numerals that do not fall under another type
Entity: Russia, Label: GPE, Countries, cities, states
Entity: Facebook, Label: ORG, Companies, agencies, institut

In [23]:

one_sentence = df['TopNews'][0]
doc = nlp(one_sentence)
spacy.displacy.render(doc, style='ent',jupyter=True)



<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h3 style="font-family: 'Verdana'; color: #FF5733;">Linguistic annotations</h3>
</div>


In [24]:
# doc should be a text here from the data

one_sentence = df['TopNews'][0]
doc = nlp(one_sentence)
for token in doc: 
    print(token.text, token.lemma_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop) 
    

North North NNP compound Xxxxx True False
Korea Korea NNP nsubj Xxxxx True False
to to TO aux xx True True
open open VB ROOT xxxx True False
its its PRP$ poss xxx True True
sky sky NN dobj xxx True False
, , , punct , False False
South south JJ amod Xxxxx True False
Korean korean JJ amod Xxxxx True False
media medium NNS compound xxxx True False
reports report NNS dobj xxxx True False
. . . punct . False False
, , , punct , False False
PTSD ptsd JJ amod XXXX True False
veterans veteran NNS nsubj xxxx True False
' ' `` case ' False False
cured cure VBN ccomp xxxx True False
' ' '' punct ' False False
with with IN prep xxxx True True
ecstasy ecstasy NN compound xxxx True False
treatment treatment NN pobj xxxx True False
in in IN prep xx True True
weeks week NNS pobj xxxx True False
, , , punct , False False
say say VBP ROOT xxx True True
scientists scientist NNS nsubj xxxx True False
, , , punct , False False
May May NNP compound Xxx True True
Day Day NNP compound Xxx True False
proteste

<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h3 style="font-family: 'Verdana'; color: #FF5733;">Morphological features with spacy </h3>
</div>


In [25]:

nlp = spacy.load("en_core_web_sm")
print("Pipeline:", nlp.pipe_names) 

sample_text = df['TopNews'][3]
doc = nlp(sample_text)
token = doc[3]

print(token)
print(token.morph)



Pipeline: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
London
Number=Sing


In [26]:
# Finding the noun chunks 
for chunk in doc.noun_chunks: 
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

Donald Trump Trump nsubj says
London hospital hospital nsubj is
a war zone zone pobj like
wounds wounds dobj stabbing
he he nsubj defends
US gun laws laws dobj defends
Nine Percent Percent nsubj Deleted
Americans Americans pobj Of
Their Facebook Accounts Accounts dobj Deleted
The Cambridge Analytica Scandal Scandal pobj Following
Zuckerberg Zuckerberg nsubj said
any meaningful impact impact attr been
new survey survey nsubj shows
17 percent percent nsubj deleted
Americans Americans pobj of
the app app dobj deleted
settings settings dobj changed
nine percent percent nsubj deleted
their accounts accounts dobj deleted
South Korea's Moon Moon nsubj says
summit summit dobj hold
U.S. U.S. pobj in
May May pobj on
Indian tea seller seller appos Moon
water water dobj using
train toilet | World news news pobj from
Kanye West West appos credits
boost boost pobj for
African-American poll numbers numbers pobj in
Scores Scores appos Moon
Aboriginal stone tools tools pobj of
Gippsland Gippsland pobj 


<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h3 style="font-family: 'Verdana'; color: #FF5733;">Generating Questions with Spacy Dependency Parsing </h3>
</div>


In [27]:

sample_spacy_text = df['TopNews'][30]

doc = nlp(sample_spacy_text) 

options = {'compact' : True, 'bg' : '#09a3d5','color': 'white', 'font': 'Trebuchet MS' } 

spacy.displacy.render(doc, jupyter=True, style='dep', options=options)


In [28]:
for token in doc: 
    print(token.text, token.head.text, token.head.pos_, [child for child in token.children])

Poland asks VERB []
asks seems VERB [Poland, Trump, establish, ., ,, is]
Trump asks VERB []
to establish VERB []
establish asks VERB [to, base, deter]
military base NOUN []
base establish VERB [military, on]
on base NOUN [border]
Russian border NOUN []
border on ADP [Russian]
to deter VERB []
deter establish VERB [to, Moscow]
Moscow deter VERB []
. asks VERB []
, asks VERB []
Avoiding is AUX [meat]
meat Avoiding VERB [and, dairy]
and meat NOUN []
dairy meat NOUN []
is asks VERB [Avoiding, way, reduce]
‘ way NOUN []
single way NOUN []
biggest way NOUN []
way is AUX [‘, single, biggest, ’]
’ way NOUN []
to reduce VERB []
reduce is AUX [to, impact]
your impact NOUN []
impact reduce VERB [your, on]
on impact NOUN [Earth]
Earth on ADP []
, seems VERB []
This seems VERB []
seems become VERB [asks, ,, This, like]
like seems VERB [impediment]
an impediment NOUN []
impediment like ADP [an, of]
of impediment NOUN [speech]
free speech NOUN []
speech of ADP [free]
; become VERB []
since become VER

In [29]:
# For generatig questions we use two parts of sentence 
from textacy.spacier import utils as spacy_utils 

def parasing_to_questions(sample_text): 
    """
    Generates a series of questions by slot filling pieces from sentences  
    
    """
    doc = nlp(spacy_sample_txt) 
    
    results = []
    
    for sentence in doc.sents: 
        
        root = sentence.root 
        
        ask_about = spacy_utils.get_subjects_of_verb(root)
        
        answers = spacy_utils.get_objects_of_verb(root)
        
        if len(ask_about) > 0 and len(answers) > 0:
            
            if root.lemma_ == "be" :
                
                question = f'What {root} {ask_about[0]}?' 
                
            else: 
                
                question = f'What does {ask_about[0]} {root.lemma_}?' 
                
                
            results.append({'question': question, 'answers':answers}) 
            
    return results 


In [30]:

spacy_sample_txt = df['TopNews'][5]

doc = nlp(spacy_sample_txt) 

spacy.displacy.render(doc, style='ent', jupyter=True)





<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <h1 style="font-family: 'Verdana'; color: #FF5733;"> Generating Questions  </h1>
</div>


In [31]:

print(parasing_to_questions(spacy_sample_txt))


[{'question': 'What does lawmakers set?', 'answers': [commission]}, {'question': 'What does panel fly?', 'answers': [flag]}, {'question': 'What does administration hire?', 'answers': [firm, discredit]}, {'question': 'What does leader want?', 'answers': [hold]}]


<h1 style="font-family: 'Lucida Sans', 'Lucida Sans Regular', 'Lucida Grande', 'Lucida Sans Unicode', Geneva, Verdana, sans-serif; text-align: center; color: #6B6E70;">Continuous Bag of Words:</h1>


<div style="background-color: #FFF3E0; padding: 20px; border-radius: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);">
    <p>
        The CBOW model architecture tries to predict the current target word (the center word) based on the source context words (surrounding words).
    </p>
    <p>
        In the CBOW model, the distributed representations of context (or surrounding words) are combined to predict the word in the middle. While in the Skip-gram model, the distributed representation of the input word is used to predict the context.

A prerequisite for any neural network or any supervised training technique is to have labeled training data. 
    </p>
</div>


In [36]:
import torch 
import torch.nn as nn
import numpy as np 

text = df['TopNews']
corpus = set(text)
corpus_length = len(corpus) 

word_dict = {}

inverse_word_dict = {}

for i, word in enumerate(corpus):
    
    word_dict[word] = i 
    
    inverse_word_dict[i] = word
    
data = []

for i in range(2, len(text) - 2): 
    
    sentence =[text[i-2], text[i-1], text[i+1], text[i+2]]
    
    target = text[i]
    
    data.append((sentence, target))
    
print(data[3])


(["Donald Trump says London hospital is 'like a war zone for stabbing wounds' as he defends US gun laws, Survey: Nine Percent Of Americans Deleted Their Facebook Accounts Following The Cambridge Analytica Scandal. Zuckerberg said there 'hasn't been any meaningful impact,' but new survey shows 17 percent of Americans deleted the app, 31 percent changed settings, nine percent deleted their accounts., South Korea's Moon, Trump to hold summit in U.S. on May 22., Indian tea seller caught using water from train toilet | World news, Trump credits Kanye West for boost in African-American poll numbers, Scores of Aboriginal stone tools discovered in Gippsland during sewer dig, Europe's biggest insurer Allianz says it will stop insuring coal-fired power plants and coal mines, 2013 Cadillac CTS Luxury Walkaround, Start up, Tour and Overview, UK Local elections: 4,000 people turned away from casting their ballot in voter ID pilot, Nico Tortorella speaks onstage throughout Rising Stars on the GLAAD 

In [42]:
# The mebedding and the continuos bag of words  

embedding_length = 20

class CBoW(torch.nn.Module):

    def __init__(self, corpus_length, embedding_dim):
        super(CBoW, self).__init__()
        
        self.embeddings = nn.Embedding(corpus_length, embedding_dim)

        self.linear1 = nn.Linear(embedding_dim, 64)
        self.linear2 = nn.Linear(64, corpus_length)
        
        self.activation_function1 = nn.ReLU()
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.LongTensor([word_dict[word]])
        return self.embeddings(word).view(1,-1)
    


In [47]:
model = CBoW(corpus_length, embedding_length)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def make_sentence_vector(sentence, word_dict):
    idxs = [word_dict[w] for w in sentence]
    return torch.tensor(idxs, dtype=torch.long)

# print(make_sentence_vector(['stormy','nights','when','the'], word_dict))


In [46]:
for epoch in range(100):
    epoch_loss = 0
    for sentence, target in data:
        model.zero_grad()
        sentence_vector = make_sentence_vector(sentence, word_dict)  
        log_probs = model(sentence_vector)
        loss = loss_function(log_probs, torch.tensor([word_dict[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.data
    print('Epoch: '+str(epoch)+', Loss: ' + str(epoch_loss.item()))

Epoch: 0, Loss: 14055.7822265625
Epoch: 1, Loss: 13411.423828125
Epoch: 2, Loss: 12790.5390625
Epoch: 3, Loss: 11763.03125
Epoch: 4, Loss: 10126.5654296875
Epoch: 5, Loss: 7918.64013671875
Epoch: 6, Loss: 5404.3125
Epoch: 7, Loss: 2996.36376953125
Epoch: 8, Loss: 1346.02490234375
Epoch: 9, Loss: 620.0885620117188
Epoch: 10, Loss: 354.03375244140625
Epoch: 11, Loss: 243.51023864746094
Epoch: 12, Loss: 195.7147979736328
Epoch: 13, Loss: 156.13470458984375
Epoch: 14, Loss: 131.58755493164062
Epoch: 15, Loss: 111.48587036132812
Epoch: 16, Loss: 96.75321197509766
Epoch: 17, Loss: 85.81900787353516
Epoch: 18, Loss: 77.36162567138672
Epoch: 19, Loss: 70.69242095947266
Epoch: 20, Loss: 65.464111328125
Epoch: 21, Loss: 61.23456573486328
Epoch: 22, Loss: 57.72932052612305
Epoch: 23, Loss: 54.728965759277344
Epoch: 24, Loss: 52.099037170410156
Epoch: 25, Loss: 49.75516891479492
Epoch: 26, Loss: 47.65483474731445
Epoch: 27, Loss: 45.73695373535156
Epoch: 28, Loss: 43.96399688720703
Epoch: 29, Loss

**Resources I used for this notebook** <br>

- Spacy Documentation
- Paketpublishing books on nlp and pytorch
- @sudalairajkumar notebook on nlp 
- @NIRANT notebook on spacy
- https://towardsdatascience.com/nlp-101-word2vec-skip-gram-and-cbow-93512ee24314
- Nlp lab blog

